# Comparing MAGs to public catalogues in MGnify, programmatically

# Import packages

[pandas](https://pandas.pydata.org/docs/reference/index.html#api) is a data analysis library with a huge list of features. It is very good at holding and manipulating table data. It is almost always short-handed to `pd`

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

`pathlib` is part of the Python standard library. We use it to find files and directories.

In [ ]:
from pathlib import Path

`time` is part of the Python standard library. We will use it to wait for results from the API.

In [ ]:
import time

`tarfile` is part of the Python standard library. We will use it to extract compressed files from a `.tar.gz` file that the API gives us.

In [ ]:
import tarfile

We need to compute a "sketch" for each Genome, using Sourmash. On the website this happens in your browser. To use the API, we do it using the [sourmash](https://sourmash.readthedocs.io/en/latest/) package. We will also use [Biopython’s SeqIO](https://biopython.org/wiki/SeqRecord) package to read the FASTA files in Python code. 

In [ ]:
import sourmash
from Bio import SeqIO


In [ ]:
holofood_mags_folder = Path('../data/salmon_mags/')

holofood_mag_files = list(holofood_mags_folder.glob('*.fa'))
holofood_mag_files

### Calculate sourmash "sketches" to search against the MGnify catalogue
We’ll compute a sourmash sketch for each MAG. 

A sketch goes into a signature, that we will use for searching. 

The signature is a sort of collection of hashes that are well suited for calculating the *containment* of your MAGs within the catalogue's MAGs.

In [ ]:
for mag_file in holofood_mag_files:
    # the sourmash parameters are chosen to match those used within MGnify
    sketch = sourmash.MinHash(n=0, ksize=31, scaled=1000)
    
    # a fasta file may have multiple records in it. add them all to the sourmash signature.
    for index, record in enumerate(SeqIO.parse(mag_file, 'fasta')):
        sketch.add_sequence(str(record.seq))

    # save the sourmash sketch as a "signature" file
    sig = sourmash.SourmashSignature(sketch, name=record.name or mag_file.stem)
    with open(mag_file.stem + '.sig', 'wt') as fp:
        sourmash.save_signatures([sig], fp)

# check what signature files we've created.
# using ! in Jupyter lets you run a shell command. It is handy for quick things like pwd and ls.
!ls *.sig

### Submit a search job to the MGnify API
We’ll call the MGnify API with all of our sketches.
There is an endpoint for this (the same one used by the website).

In this case, we need to **send** data to the API (not just fetch it). This is called "POST"ing data in the API world. 

This part of the API is quite specialized and so is not a formal JSON:API, so we use the more flexible [requests](https://docs.python-requests.org/en/master/) Python package to communicate with it.

In [ ]:
import requests

We're going to compare the MAGs against the human gut catalogue. This might seem an odd choice, but the UHGG catalogue is by far the most comprehensive catalogue of MAGs in any biome.

In [ ]:
endpoint = 'https://www.ebi.ac.uk/metagenomics/api/v1/genomes-search/gather'
catalogue_id = 'human-gut-v2-0'  # You could change this to any other catalogue ID from the MGnify website, if you use this in the future.

# Create a list of file uploads, and attach them to the API request
signatures = [open(mag.stem + '.sig', 'rb') for mag in holofood_mag_files]
sketch_uploads = [('file_uploaded', signature) for signature in signatures]

# Send the API request - it specifies which catalogue to search against and attaches all of the signature files.
submitted_job = requests.post(endpoint, data={'mag_catalog': catalogue_id}, files=sketch_uploads).json()

map(lambda fp: fp.close(), signatures)  # tidy up open file pointers

print(submitted_job)

### Wait for our results to be ready
As you can see in the printed `submitted_job` above, a `status_URL` was returned in the response from submitting the job via the API.
Since the job will be in a queue, we must poll this `status_URL` to wait for our job to be completed.
We’ll check every 2 seconds until ALL of the jobs are finished.

In [ ]:
job_done = False
while not job_done:
    print('Checking status...')
    status = requests.get(submitted_job['data']['status_URL'])
    # the status_URL is just another API endpoint that's unique for our search job
    
    queries_done = {sig['job_id']: sig['status'] for sig in status.json()['data']['signatures']}
    job_done = all(map(lambda q: q == 'SUCCESS', queries_done.values()))
    if not job_done:
        print('Still waiting for jobs to complete. Current status of jobs')
        print(queries_done)
        print('Will check again in 2 seconds')
        time.sleep(2)

print('All finished!')

<div class="alert alert-block alert-warning">
    If you happen to get an error at this point, just run the cell again. If lots of people are using the API at the same time it can sometimes struggle.
</div>

### See if we got any matches


<div class="alert alert-block alert-info">
    Coding challenge!
</div>

`status` is a JSON object. You can use the `status.json()` method to read the JSON into a Python `dict`.

It'll look something like this:
```json
{'data': {'group_id': '...',
  'signatures': [{
    'status': 'SUCCESS',
    'filename': '...',
    'result': {
        'status': 'NO_RESULTS',
        'catalog': 'human-gut-v2-0',
        'query_filename': '...'
    }
   },
   {'job_id': '...',
    'status': 'SUCCESS',
    'filename': 'ERR4918all_bin.2.sig',
    'result': {'overlap': '0.9 Mbp',
     'p_query': '77.8%',
     'p_match': '55.2%',
     'match': 'MGYG000000001',
     'catalog': 'human-gut-v2-0',
     'query_filename': '...'},
   }]
}
```

I.e., `status.json()` contains a list of results in `['data']['signatures']`.
Each object in that list has a `['signatures']` key with a value that is another dictionary.

Write code to find any objects values in `data.signatures` that have a `result.match` entry.

In [ ]:
matches = 

##### Solution
Unhide the following cell for a solution

In [ ]:
matches = [sig for sig in status.json()['data']['signatures'] if 'match' in sig['result']]
matches

<div class="alert alert-block alert-info">
    Coding challenge!
</div>

Lets use the MGnify API to find out more about the matched MAGs.

Complete this code to call the MGnify API endpoint `https://www.ebi.ac.uk/metagenomics/api/v1/genomes/MGYGxxxxxxxxx` for each `MGYGxx...` accession, using a GET request.
The `MGYGxxx...` is in `matches[i]['result']['match']`.
You'll get back JSON data. Save the `data.attributes` object (that's all the useful info about the MAG) to each match, as `matches[i]['mgnify_match_details'].

In [ ]:
endpoint = 'https...............

for match in matches:
    mgyg_in_public_catalogue = match['............
                                     
    mag_details_response = requests.get(f'.........................
    match['mgnify_match_details'] = .....................

##### Solution
Unhide the cell below for the full solution

In [ ]:
endpoint = 'https://www.ebi.ac.uk/metagenomics/api/v1/genomes'

for match in matches:
    mgyg_in_public_catalogue = match['result']['match']
    mag_details_response = requests.get(f'{endpoint}/{mgyg_in_public_catalogue}')
    match['mgnify_match_details'] = mag_details_response.json()['data']['attributes']

Put all of the results into a table (dataframe):

In [ ]:
matches_table = pd.json_normalize(matches)
matches_table

#### In slightly more readable form...

In [ ]:
matches_table[['filename', 'result.match', 'mgnify_match_details.taxon-lineage', 'result.p_query']]

### What taxonomic lineages do we have in the matches?

<div class="alert alert-block alert-info">
    Coding challenge!
</div>

Use the Pandas `.unique()` method to list all of the taxonomic lineages from any of our matches:

##### Solution
Unhide the cell below for a solution

In [ ]:
matches_table['mgnify_match_details.taxon-lineage'].unique()